In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
import random as rn
from datetime import datetime
import warnings
import pickle
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# 시드 설정
RANDOM_SEED = 2025
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)

# XGBoost 버전 확인
print(f"XGBoost 버전: {xgboost.__version__}")

print("🚀 전력 사용량 예측 앙상블 모델 시작!")
print("=" * 50)


XGBoost 버전: 1.6.1
🚀 전력 사용량 예측 앙상블 모델 시작!


In [2]:
def smape(gt, preds):
    """SMAPE (Symmetric Mean Absolute Percentage Error) 계산"""
    gt = np.array(gt)
    preds = np.array(preds)
    v = 2 * abs(preds - gt) / (abs(preds) + abs(gt))
    score = np.mean(v) * 100
    return score
    
def weighted_mse(alpha=1):
    """가중 MSE 손실 함수 (Under-prediction에 더 큰 페널티)"""
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual > 0, -2 * alpha * residual, -2 * residual)
        hess = np.where(residual > 0, 2 * alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def custom_smape(preds, dtrain):
    """XGBoost용 SMAPE 평가 함수"""
    labels = dtrain.get_label()
    return 'custom_smape', np.mean(2 * abs(preds - labels) / (abs(preds) + abs(labels))) * 100

print("✅ 평가 함수 정의 완료")


✅ 평가 함수 정의 완료


In [3]:
# 데이터 로드
print("📊 데이터 로드 중...")
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
building_info = pd.read_csv('data/building_info.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

print(f"✅ Train 데이터: {train.shape}")
print(f"✅ Test 데이터: {test.shape}")
print(f"✅ Building info: {building_info.shape}")

# 컬럼명 영어로 변경 (작년 수상자 방식)
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train.drop('num_date_time', axis=1, inplace=True)

test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation'
})
test.drop('num_date_time', axis=1, inplace=True)

building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

# 건물 유형 영어로 번역
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '학교': 'University',
    '백화점': 'Department Store',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    'IDC(전화국)': 'IDC',
    '호텔': 'Hotel'
}
building_info['building_type'] = building_info['building_type'].replace(translation_dict)

# 태양광/ESS 설비 유무 피처 생성
building_info['solar_power_utility'] = np.where(building_info.solar_power_capacity != '-', 1, 0)
building_info['ess_utility'] = np.where(building_info.ess_capacity != '-', 1, 0)

# 건물 정보 병합
train = pd.merge(train, building_info, on='building_number', how='left')
test = pd.merge(test, building_info, on='building_number', how='left')

print("✅ 기본 전처리 완료")


📊 데이터 로드 중...
✅ Train 데이터: (204000, 10)
✅ Test 데이터: (16800, 7)
✅ Building info: (100, 7)
✅ 기본 전처리 완료


In [4]:
# 날짜/시간 변환 및 기본 시간 피처 생성
print("🔧 Feature Engineering 시작...")

train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')
test['date_time'] = pd.to_datetime(test['date_time'], format='%Y%m%d %H')

# 기본 시간 피처
for df in [train, test]:
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['day_of_week'] = df['date_time'].dt.dayofweek

print("✅ 기본 시간 피처 생성 완료")


🔧 Feature Engineering 시작...
✅ 기본 시간 피처 생성 완료


In [5]:
# 일별 온도 통계 피처 생성
def calculate_day_values(dataframe, target_column, output_column, aggregation_func):
    """일별 통계값 계산 함수"""
    result_dict = {}
    grouped_temp = dataframe.groupby(['building_number', 'month', 'day'])[target_column].agg(aggregation_func)
    
    for (building, month, day), value in grouped_temp.items():
        result_dict.setdefault(building, {}).setdefault(month, {})[day] = value
    
    dataframe[output_column] = [
        result_dict.get(row['building_number'], {}).get(row['month'], {}).get(row['day'], None)
        for _, row in dataframe.iterrows()
    ]

# 일별 온도 통계 피처 생성
for df in [train, test]:
    calculate_day_values(df, 'temperature', 'day_max_temperature', 'max')
    calculate_day_values(df, 'temperature', 'day_mean_temperature', 'mean')
    calculate_day_values(df, 'temperature', 'day_min_temperature', 'min')
    df['day_temperature_range'] = df['day_max_temperature'] - df['day_min_temperature']

print("✅ 일별 온도 통계 피처 생성 완료")


✅ 일별 온도 통계 피처 생성 완료


In [6]:
# 이상치 제거 및 추가 피처 생성
outlier_idx = train.index[train['power_consumption'] == 0].tolist()
print(f"제거할 이상치 개수: {len(outlier_idx)}")
train.drop(index=outlier_idx, inplace=True)
print(f"남은 행 개수: {train.shape[0]}")

# 공휴일 피처 생성
holi_weekday = ['2024-06-06', '2024-08-15']
train['holiday'] = np.where(
    (train.day_of_week >= 5) | (train.date_time.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0
)
test['holiday'] = np.where(
    (test.day_of_week >= 5) | (test.date_time.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0
)



# 주기성 피처 생성 (Cyclical Features)
for df in [train, test]:
    # 시간 주기성
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 23.0)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 23.0)
    
    # 날짜 주기성
    df['sin_date'] = -np.sin(2 * np.pi * (df['month'] + df['day'] / 31) / 12)
    df['cos_date'] = -np.cos(2 * np.pi * (df['month'] + df['day'] / 31) / 12)
    
    # 월 주기성
    df['sin_month'] = -np.sin(2 * np.pi * df['month'] / 12.0)
    df['cos_month'] = -np.cos(2 * np.pi * df['month'] / 12.0)
    
    # 요일 주기성
    df['sin_dayofweek'] = -np.sin(2 * np.pi * (df['day_of_week'] + 1) / 7.0)
    df['cos_dayofweek'] = -np.cos(2 * np.pi * (df['day_of_week'] + 1) / 7.0)

print("✅ 이상치 제거 및 주기성 피처 생성 완료")


제거할 이상치 개수: 68
남은 행 개수: 203932
✅ 이상치 제거 및 주기성 피처 생성 완료


In [ ]:

def create_inferred_holidays(df, building_info):
    # 규칙이 없는 백화점 건물 번호 추출
    department_store_buildings = [19,34,45,54,73,74,79,88,95]

    # 백화점 건물에 대해 추정 휴일만 적용
    for building_num in department_store_buildings:
        building_data = df[df['building_number'] == building_num].copy()

        if len(building_data) > 0:
            # 일별 전력소비량 계산
            building_data['date'] = building_data['date_time'].dt.date
            daily_consumption = building_data.groupby('date')['power_consumption'].sum()

            # 전체 평균 기준 임계치 (0.7배)
            threshold = daily_consumption.mean() * 0.7
            inferred_holiday_dates = daily_consumption[daily_consumption < threshold].index

            # 추정 휴일을 해당 건물의 holiday 컬럼에 적용
            for holiday_date in inferred_holiday_dates:
                mask = (df['building_number'] == building_num) & (df['date_time'].dt.date == holiday_date)
                df.loc[mask, 'holiday'] = 1

    return df

# 휴일 피처 생성
train = create_inferred_holidays(train, building_info)

In [8]:
# 매주 일요일 휴무 건물(18번) 추가 및 기존 규칙 유지

def apply_specific_building_holidays(df):
    """
    특정 건물들의 휴일 규칙을 적용하는 함수 (달력 기준 주차 계산)
    - 18번 건물: 매주 일요일 휴무
    - 27, 40, 63번 건물: 홀수 주 일요일 휴무
    - 29번 건물: 매달 10일 + 5번째 주 일요일 휴무
    - 32번 건물: 홀수 주 월요일 휴무
    """
    print("🔧 특정 건물 휴일 규칙 적용 중... (달력 기준 주차)")
    department_store_buildings = building_info[building_info['building_type'] == 'Department Store']['building_number'].tolist()

    # 백화점 건물에 해당하는 데이터만 holiday를 0으로 초기화
    df.loc[df['building_number'].isin(department_store_buildings), 'holiday'] = 0

    # 매주 일요일 휴무 건물 (18번)
    every_sunday_buildings = [18]

    # 홀수 주 일요일 휴무 건물들 (27, 40, 59, 63)
    odd_week_sunday_buildings = [27, 40, 59, 63]

    # 매달 10일 + 5번째 주 일요일 휴무 건물 (29)
    special_holiday_building = [29]

    # 홀수 주 월요일 휴무 건물 (32)
    odd_week_monday_buildings = [32]

    # 달력 기준 주차 계산 (일요일 시작)
    def get_calendar_week_of_month(date_series):
        """달력 기준 주차 계산 (월의 첫날이 포함된 주를 1주차로)"""
        result = []
        for date in date_series:
            first_day = date.replace(day=1)
            first_day_weekday = first_day.weekday()
            days_to_week_start = (first_day_weekday + 1) % 7  # 일요일 기준
            week_start_of_first = first_day - pd.Timedelta(days=days_to_week_start)
            days_since_first_week_start = (date - week_start_of_first).days
            week_num = (days_since_first_week_start // 7) + 1
            result.append(week_num)
        return result

    # 달력 기준 주차 계산
    df['calendar_week_of_month'] = get_calendar_week_of_month(df['date_time'])

    # 0. 매주 일요일 휴무 (18번 건물)
    for building_num in every_sunday_buildings:
        mask = (df['building_number'] == building_num) & (df['day_of_week'] == 6)
        df.loc[mask, 'holiday'] = 1
        count = mask.sum()
        print(f"   건물 {building_num}: 매주 일요일 휴일 {count}개 적용")

    # 1. 홀수 주 일요일 휴무 (27, 40, 59, 63번 건물)
    for building_num in odd_week_sunday_buildings:
        mask = (df['building_number'] == building_num) & \
               (df['day_of_week'] == 6) & \
               (df['calendar_week_of_month'] % 2 == 1)
        df.loc[mask, 'holiday'] = 1
        count = mask.sum()
        print(f"   건물 {building_num}: 홀수 주 일요일 휴일 {count}개 적용")

    # 2. 매달 10일 + 5번째 주 일요일 휴무 (29번 건물)
    for building_num in special_holiday_building:
        mask_10th = (df['building_number'] == building_num) & (df['date_time'].dt.day == 10)
        df.loc[mask_10th, 'holiday'] = 1
        count_10th = mask_10th.sum()
        mask_5th_sunday = (df['building_number'] == building_num) & \
                          (df['day_of_week'] == 6) & \
                          (df['calendar_week_of_month'] == 5)
        df.loc[mask_5th_sunday, 'holiday'] = 1
        count_5th_sunday = mask_5th_sunday.sum()
        print(f"   건물 {building_num}: 매달 10일 휴일 {count_10th}개, 5번째 주 일요일 {count_5th_sunday}개 적용")

    # 3. 홀수 주 월요일 휴무 (32번 건물)
    for building_num in odd_week_monday_buildings:
        mask = (df['building_number'] == building_num) & \
               (df['day_of_week'] == 0) & \
               (df['calendar_week_of_month'] % 2 == 1)
        df.loc[mask, 'holiday'] = 1
        count = mask.sum()
        print(f"   건물 {building_num}: 홀수 주 월요일 휴일 {count}개 적용")

    # 임시 컬럼 제거
    df.drop(['calendar_week_of_month'], axis=1, inplace=True)

    return df

# test 데이터에 특정 건물 휴일 규칙 적용
test = apply_specific_building_holidays(test)

print("✅ test 데이터에 백화점 추정 휴일 적용 완료")
print("✅ test 데이터에 특정 건물 휴일 규칙 적용 완료")


🔧 특정 건물 휴일 규칙 적용 중... (달력 기준 주차)
   건물 18: 매주 일요일 휴일 24개 적용
   건물 27: 홀수 주 일요일 휴일 24개 적용
   건물 40: 홀수 주 일요일 휴일 24개 적용
   건물 59: 홀수 주 일요일 휴일 24개 적용
   건물 63: 홀수 주 일요일 휴일 24개 적용
   건물 29: 매달 10일 휴일 0개, 5번째 주 일요일 24개 적용
   건물 32: 홀수 주 월요일 휴일 24개 적용
✅ test 데이터에 백화점 추정 휴일 적용 완료
✅ test 데이터에 특정 건물 휴일 규칙 적용 완료


In [9]:
# 특정 건물들의 휴일 적용 결과 확인
print("🔍 특정 건물들의 휴일 적용 결과 확인:")

# 27번 건물 (홀수 주 일요일) 휴일 확인
holiday_dates_27 = test.loc[(test['building_number'] == 27) & (test['holiday'] == 1), 'date_time'].dt.date
unique_holiday_dates_27 = holiday_dates_27.drop_duplicates().tolist()
print(f"건물 27번 (홀수 주 일요일) 휴일: {len(unique_holiday_dates_27)}일")
print(f"날짜: {unique_holiday_dates_27[:10]}...")  # 처음 10개만 출력

# 29번 건물 (매달 10일 + 5번째 주 일요일) 휴일 확인  
holiday_dates_29 = test.loc[(test['building_number'] == 29) & (test['holiday'] == 1), 'date_time'].dt.date
unique_holiday_dates_29 = holiday_dates_29.drop_duplicates().tolist()
print(f"건물 29번 (매달 10일 + 5번째 주 일요일) 휴일: {len(unique_holiday_dates_29)}일")
print(f"날짜: {unique_holiday_dates_29[:10]}...")  # 처음 10개만 출력

# 32번 건물 (홀수 주 월요일) 휴일 확인
holiday_dates_32 = test.loc[(test['building_number'] == 32) & (test['holiday'] == 1), 'date_time'].dt.date
unique_holiday_dates_32 = holiday_dates_32.drop_duplicates().tolist()
print(f"건물 32번 (홀수 주 월요일) 휴일: {len(unique_holiday_dates_32)}일")
print(f"날짜: {unique_holiday_dates_32[:10]}...")  # 처음 10개만 출력

🔍 특정 건물들의 휴일 적용 결과 확인:
건물 27번 (홀수 주 일요일) 휴일: 1일
날짜: [datetime.date(2024, 8, 25)]...
건물 29번 (매달 10일 + 5번째 주 일요일) 휴일: 1일
날짜: [datetime.date(2024, 8, 25)]...
건물 32번 (홀수 주 월요일) 휴일: 1일
날짜: [datetime.date(2024, 8, 26)]...


In [10]:
n  = 59
holiday_dates_n = test.loc[(test['building_number'] == n) & (test['holiday'] == 1), 'date_time'].dt.date
unique_holiday_dates_n = holiday_dates_n.drop_duplicates().tolist()
print(f"건물 {n}번 (홀수 주 월요일) 휴일: {len(unique_holiday_dates_n)}일")
print(f"날짜: {unique_holiday_dates_n[:10]}...")  # 처음 10개만 출력

건물 59번 (홀수 주 월요일) 휴일: 1일
날짜: [datetime.date(2024, 8, 25)]...


In [11]:
# 기상 관련 파생 피처 생성
def CDH(xs):
    """Cooling Degree Hours 계산"""
    cumsum = np.cumsum(xs - 26)
    return np.concatenate((cumsum[:11], cumsum[11:] - cumsum[:-11]))

def calculate_and_add_cdh(dataframe):
    """건물별 CDH 계산 및 추가"""
    cdhs = []
    for i in range(1, 101):
        temp = dataframe[dataframe['building_number'] == i]['temperature'].values
        cdh = CDH(temp)
        cdhs.append(cdh)
    return np.concatenate(cdhs)

# CDH, THI, WCT 피처 생성
train['CDH'] = calculate_and_add_cdh(train)
test['CDH'] = calculate_and_add_cdh(test)

# THI (Temperature Humidity Index)
train['THI'] = 9/5 * train['temperature'] - 0.55 * (1 - train['humidity']/100) * (9/5 * train['temperature'] - 26) + 32
test['THI'] = 9/5 * test['temperature'] - 0.55 * (1 - test['humidity']/100) * (9/5 * test['temperature'] - 26) + 32

# WCT (Wind Chill Temperature)
train['WCT'] = 13.12 + 0.6125 * train['temperature'] - 11.37 * (train['windspeed']**0.16) + 0.3965 * (train['windspeed']**0.16) * train['temperature']
test['WCT'] = 13.12 + 0.6125 * test['temperature'] - 11.37 * (test['windspeed']**0.16) + 0.3965 * (test['windspeed']**0.16) * test['temperature']

print("✅ 기상 관련 파생 피처 생성 완료")


✅ 기상 관련 파생 피처 생성 완료


In [12]:
# 전력 소비량 기반 통계 피처 생성 (Target-like Features)
print("📊 전력 소비량 기반 통계 피처 생성 중...")

# 건물별 시간대/요일별 평균 및 표준편차
power_mean = pd.pivot_table(train, values='power_consumption', 
                           index=['building_number', 'hour', 'day_of_week'], 
                           aggfunc=np.mean).reset_index()
power_mean.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_mean']

power_std = pd.pivot_table(train, values='power_consumption', 
                          index=['building_number', 'hour', 'day_of_week'], 
                          aggfunc=np.std).reset_index()
power_std.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_std']

# 건물별 시간대별 평균 및 표준편차
power_hour_mean = pd.pivot_table(train, values='power_consumption', 
                                index=['building_number', 'hour'], 
                                aggfunc=np.mean).reset_index()
power_hour_mean.columns = ['building_number', 'hour', 'hour_mean']

power_hour_std = pd.pivot_table(train, values='power_consumption', 
                               index=['building_number', 'hour'], 
                               aggfunc=np.std).reset_index()
power_hour_std.columns = ['building_number', 'hour', 'hour_std']

# 통계 피처 병합
train = train.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')
train = train.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')
train = train.merge(power_hour_mean, on=['building_number', 'hour'], how='left')
train = train.merge(power_hour_std, on=['building_number', 'hour'], how='left')

test = test.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_hour_mean, on=['building_number', 'hour'], how='left')
test = test.merge(power_hour_std, on=['building_number', 'hour'], how='left')

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

print("✅ 전력 소비량 기반 통계 피처 생성 완료")
print(f"최종 train 데이터 shape: {train.shape}")
print(f"최종 test 데이터 shape: {test.shape}")


📊 전력 소비량 기반 통계 피처 생성 중...
✅ 전력 소비량 기반 통계 피처 생성 완료
최종 train 데이터 shape: (203932, 41)
최종 test 데이터 shape: (16800, 38)


In [13]:
# 모델링용 피처 선택 (작년 수상자 방식)
drop_columns = [
    'solar_power_capacity', 'ess_capacity', 'pcs_capacity',
    'power_consumption', 'rainfall', 'sunshine', 'solar_radiation',
    'hour', 'day', 'month', 'day_of_week', 'date_time'
]

X = train.drop(drop_columns, axis=1)
Y = train[['building_type', 'power_consumption']]
test_X = test.drop([col for col in drop_columns if col in test.columns], axis=1)

print(f"✅ 모델링용 데이터 준비 완료")
print(f"피처 수: {X.shape[1]}")
print(f"건물 유형 수: {len(X['building_type'].unique())}")
print(f"건물 수: {len(X['building_number'].unique())}")

# 건물 유형 리스트
type_list = X["building_type"].unique()
building_list = X["building_number"].unique()

print(f"건물 유형: {type_list}")
print(f"건물 번호 범위: {min(building_list)} ~ {max(building_list)}")


✅ 모델링용 데이터 준비 완료
피처 수: 29
건물 유형 수: 10
건물 수: 100
건물 유형: ['Hotel' 'Commercial' 'Hospital' 'University' 'Other Buildings'
 'Apartment' 'Research Institute' 'Department Store' 'IDC' 'Public']
건물 번호 범위: 1 ~ 100


In [14]:
## 백화점만 train
print("🏢 Department Store 모델 훈련 시작...")
print("=" * 50)

KFOLD_SPLITS = 7
kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_SEED)

# 백화점 전용 결과 저장용 DataFrame
dept_mask_test = test_X['building_number'].isin(building_info[building_info['building_type'] == 'Department Store']['building_number'])
dept_mask_train = X['building_number'].isin(building_info[building_info['building_type'] == 'Department Store']['building_number'])

type_model_predictions = pd.DataFrame(index=test_X[dept_mask_test].index, columns=["answer"], dtype=float)
type_model_oof = pd.DataFrame(index=X[dept_mask_train].index, columns=["pred"], dtype=float)

type_model_scores = {}

btype = "Department Store"
print(f"\n🔍 건물 유형: {btype}")

# 해당 유형 데이터 필터링
x = X[X['building_type'] == btype].copy()
y = Y[Y['building_type'] == btype]['power_consumption'].copy()
xt = test_X[test_X['building_type'] == btype].copy()

print(f"   📊 훈련 데이터: {len(x)}개")
print(f"   📊 테스트 데이터: {len(xt)}개")

# 건물 번호 원-핫 인코딩
x = pd.get_dummies(x, columns=["building_number"], drop_first=False)
xt = pd.get_dummies(xt, columns=["building_number"], drop_first=False)

# 테스트 데이터에 없는 컬럼 처리
xt = xt.reindex(columns=x.columns, fill_value=0)

# building_type 컬럼 제거
x = x.drop(columns=["building_type"])
xt = xt.drop(columns=["building_type"])

# K-Fold 교차 검증
preds_valid = pd.Series(index=y.index, dtype=float)
preds_test = []
fold_scores = []

x_values = x.values
y_values = y.values

for fold, (tr_idx, va_idx) in enumerate(kf.split(x_values), 1):
    X_tr, X_va = x_values[tr_idx], x_values[va_idx]
    y_tr, y_va = y_values[tr_idx], y_values[va_idx]
    
    # 로그 변환
    y_tr_log = np.log(y_tr)
    y_va_log = np.log(y_va)
    
    # XGBoost 모델 훈련
    model = XGBRegressor(
        learning_rate=0.05,
        n_estimators=5000,
        max_depth=10,
        subsample=0.7,
        colsample_bytree=0.5,
        min_child_weight=3,
        random_state=RANDOM_SEED,
        objective=weighted_mse(3),
        early_stopping_rounds=100,
    )
    
    model.fit(
        X_tr, y_tr_log,
        eval_set=[(X_va, y_va_log)],
        eval_metric=custom_smape,
        verbose=False,
    )
    
    # 검증 예측 (로그 역변환)
    va_pred = np.exp(model.predict(X_va))
    preds_valid.iloc[va_idx] = va_pred
    
    # 성능 계산
    fold_smape = smape(y_va, va_pred)
    fold_scores.append(fold_smape)
    
    # 테스트 예측
    preds_test.append(np.exp(model.predict(xt.values)))

# 검증 예측 저장
type_model_oof.loc[preds_valid.index, "pred"] = preds_valid

# 테스트 예측 (앙상블 평균) 저장
type_model_predictions.loc[xt.index, "answer"] = np.mean(preds_test, axis=0)

# 성능 저장
avg_smape = np.mean(fold_scores)
type_model_scores[btype] = avg_smape

print(f"   🏆 평균 SMAPE: {avg_smape:.4f}")

# 백화점 성능 계산 (NaN 값 제거)
dept_y_true = Y[dept_mask_train]["power_consumption"].values
dept_y_pred = type_model_oof["pred"].values

# NaN 값 제거
valid_mask = ~(pd.isna(dept_y_true) | pd.isna(dept_y_pred))
if np.sum(valid_mask) > 0:
    total_type_smape = smape(dept_y_true[valid_mask], dept_y_pred[valid_mask])
else:
    total_type_smape = float('nan')

print(f"\n🎯 Department Store 모델 전체 SMAPE: {total_type_smape:.4f}")
print("✅ Department Store 모델 훈련 완료")


🏢 Department Store 모델 훈련 시작...

🔍 건물 유형: Department Store
   📊 훈련 데이터: 32636개
   📊 테스트 데이터: 2688개
   🏆 평균 SMAPE: 3.6176

🎯 Department Store 모델 전체 SMAPE: 3.6176
✅ Department Store 모델 훈련 완료


In [15]:
## 백화점 만 train

print("🏠 건물별 개별 모델 훈련 시작...")
print("=" * 50)

# 'Department Store' 건물 번호만 추출
department_store_buildings = building_info[building_info['building_type'] == 'Department Store']['building_number'].tolist()

# 백화점 전용 결과 저장용 DataFrame (기존에 정의된 마스크 재사용)
individual_model_predictions = pd.DataFrame(index=test_X[dept_mask_test].index, columns=["answer"], dtype=float)
individual_model_oof = pd.DataFrame(index=X[dept_mask_train].index, columns=["pred"], dtype=float)

individual_model_scores = {}

for building_num in sorted(department_store_buildings):
    print(f"\n🏢 건물 {building_num} 훈련 중...")
    
    # 해당 건물 데이터 필터링
    x = X[X['building_number'] == building_num].copy()
    y = Y[Y.index.isin(x.index)]['power_consumption'].copy()
    xt = test_X[test_X['building_number'] == building_num].copy()
    
    print(f"   📊 훈련 데이터: {len(x)}개")
    print(f"   📊 테스트 데이터: {len(xt)}개")
    
    # 불필요한 컬럼 제거 (건물번호, 건물유형)
    feature_cols = [col for col in x.columns if col not in ['building_number', 'building_type']]
    x_features = x[feature_cols].copy()
    xt_features = xt[feature_cols].copy()
    
    # K-Fold 교차 검증
    preds_valid = pd.Series(index=y.index, dtype=float)
    preds_test = []
    fold_scores = []
    
    x_values = x_features.values
    y_values = y.values
    
    for fold, (tr_idx, va_idx) in enumerate(kf.split(x_values), 1):
        X_tr, X_va = x_values[tr_idx], x_values[va_idx]
        y_tr, y_va = y_values[tr_idx], y_values[va_idx]
        
        # 로그 변환
        y_tr_log = np.log(y_tr)
        y_va_log = np.log(y_va)
        
        # XGBoost 모델 훈련
        model = XGBRegressor(
            learning_rate=0.05,
            n_estimators=5000,
            max_depth=10,
            subsample=0.7,
            colsample_bytree=0.5,
            min_child_weight=3,
            random_state=RANDOM_SEED,
            objective=weighted_mse(3),
            early_stopping_rounds=100,
        )
        
        model.fit(
            X_tr, y_tr_log,
            eval_set=[(X_va, y_va_log)],
            eval_metric=custom_smape,
            verbose=False,
        )
        
        # 검증 예측 (로그 역변환)
        va_pred = np.exp(model.predict(X_va))
        preds_valid.iloc[va_idx] = va_pred
        
        # 성능 계산
        fold_smape = smape(y_va, va_pred)
        fold_scores.append(fold_smape)
        
        # 테스트 예측
        preds_test.append(np.exp(model.predict(xt_features.values)))
    
    # 검증 예측 저장
    individual_model_oof.loc[preds_valid.index, "pred"] = preds_valid
    
    # 테스트 예측 (앙상블 평균) 저장
    individual_model_predictions.loc[xt.index, "answer"] = np.mean(preds_test, axis=0)
    
    # 성능 저장
    avg_smape = np.mean(fold_scores)
    individual_model_scores[building_num] = avg_smape
    
    print(f"   🏆 평균 SMAPE: {avg_smape:.4f}")
    
    # 진행률 출력 (10개마다)
    if building_num % 10 == 0:
        progress = building_num / len(department_store_buildings) * 100
        avg_score = np.mean(list(individual_model_scores.values()))
        print(f"   ⏳ 진행률: {progress:.1f}% | 평균 SMAPE: {avg_score:.4f}")

# 백화점 개별 모델 성능 계산 (NaN 값 제거)
dept_y_true_individual = Y[dept_mask_train]["power_consumption"].values
dept_y_pred_individual = individual_model_oof["pred"].values

# NaN 값 제거
valid_mask_individual = ~(pd.isna(dept_y_true_individual) | pd.isna(dept_y_pred_individual))
if np.sum(valid_mask_individual) > 0:
    total_individual_smape = smape(dept_y_true_individual[valid_mask_individual], dept_y_pred_individual[valid_mask_individual])
else:
    total_individual_smape = float('nan')

print(f"\n🎯 건물별 개별 모델 전체 SMAPE: {total_individual_smape:.4f}")
print("✅ 건물별 개별 모델 훈련 완료")


🏠 건물별 개별 모델 훈련 시작...

🏢 건물 18 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 3.1663

🏢 건물 19 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 4.6571

🏢 건물 27 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 3.1070

🏢 건물 29 훈련 중...
   📊 훈련 데이터: 2037개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 1.9587

🏢 건물 32 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 3.3886

🏢 건물 34 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 1.9461

🏢 건물 40 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 2.9426
   ⏳ 진행률: 250.0% | 평균 SMAPE: 3.0238

🏢 건물 45 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 3.3811

🏢 건물 54 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 6.4161

🏢 건물 59 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 3.8745

🏢 건물 63 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 2.7206

🏢 건물 73 훈련 중...
   📊 훈련 데이터: 2040개
   📊 테스트 데이터: 168개
   🏆 평균 SMAPE: 2.8266

🏢 건물 74 훈련 중...
  

In [16]:
print("🎯 백화점 앙상블 모델 + 기존 결과 결합 중...")
print("=" * 50)

# 1. 기존 앙상블 결과 로드
print("📂 기존 앙상블 결과 로드 중...")
existing_submission = pd.read_csv('result_csv/ensemble_submission.csv')
print(f"✅ 기존 제출 파일 로드 완료: {existing_submission.shape}")

# 2. 백화점 건물 번호 추출
department_store_buildings = building_info[building_info['building_type'] == 'Department Store']['building_number'].tolist()
print(f"🏢 백화점 건물 번호: {sorted(department_store_buildings)}")

# 3. 백화점에 대한 앙상블 가중치 설정
individual_weight = 0.7  # 개별 건물 모델 가중치
type_weight = 0.3        # 건물 타입별 모델 가중치

print(f"\n📊 백화점 앙상블 가중치:")
print(f"   개별 건물 모델: {individual_weight * 100}%")
print(f"   건물 타입별 모델: {type_weight * 100}%")

# 4. 백화점 건물에 대해서만 앙상블 수행
department_store_mask = test_X['building_number'].isin(department_store_buildings)
department_train_mask = X['building_number'].isin(department_store_buildings)

# 데이터 상태 확인
print(f"📊 앙상블 전 데이터 상태 확인:")
print(f"   개별 모델 예측값 개수: {len(individual_model_predictions)}")
print(f"   타입별 모델 예측값 개수: {len(type_model_predictions)}")
print(f"   개별 모델 OOF 예측값 개수: {len(individual_model_oof)}")
print(f"   타입별 모델 OOF 예측값 개수: {len(type_model_oof)}")

# 백화점 검증 데이터 앙상블 (성능 평가용)
ensemble_oof_dept = (
    individual_model_oof["pred"] * individual_weight + 
    type_model_oof["pred"] * type_weight
)

# 백화점 테스트 데이터 앙상블
ensemble_predictions_dept = (
    individual_model_predictions["answer"] * individual_weight + 
    type_model_predictions["answer"] * type_weight
)

print(f"   앙상블 후 검증 예측값 개수: {len(ensemble_oof_dept)}")
print(f"   앙상블 후 테스트 예측값 개수: {len(ensemble_predictions_dept)}")

# 5. 최종 제출 파일 생성 (백화점은 새로운 값, 나머지는 기존 값)
final_submission = existing_submission.copy()

# 백화점 건물에 해당하는 인덱스 찾기
dept_test_indices = test_X[department_store_mask].index
print(f"📊 백화점 테스트 데이터 인덱스 수: {len(dept_test_indices)}")

# 백화점 건물 예측값을 새로운 앙상블 결과로 교체
print(f"📊 예측값 교체 과정:")
print(f"   백화점 테스트 인덱스 범위: {dept_test_indices.min()} ~ {dept_test_indices.max()}")
print(f"   앙상블 예측값 인덱스 범위: {ensemble_predictions_dept.index.min()} ~ {ensemble_predictions_dept.index.max()}")

replacement_count = 0
for idx in dept_test_indices:
    if idx < len(final_submission) and idx in ensemble_predictions_dept.index:
        if pd.notna(ensemble_predictions_dept.loc[idx]):
            final_submission.loc[idx, 'answer'] = ensemble_predictions_dept.loc[idx]
            replacement_count += 1
        else:
            print(f"   ⚠️ 인덱스 {idx}의 예측값이 NaN입니다.")
    else:
        print(f"   ⚠️ 인덱스 {idx}를 찾을 수 없습니다.")

print(f"   성공적으로 교체된 예측값: {replacement_count}개")

# 6. 백화점 성능 계산
# 백화점 훈련 데이터의 실제값과 앙상블 예측값 추출
dept_train_indices = X[department_train_mask].index
dept_y_true = Y.loc[dept_train_indices, "power_consumption"].values
dept_y_pred = ensemble_oof_dept.values

print(f"\n📊 성능 계산을 위한 데이터 확인:")
print(f"   백화점 훈련 인덱스 수: {len(dept_train_indices)}")
print(f"   실제값 개수: {len(dept_y_true)}")
print(f"   예측값 개수: {len(dept_y_pred)}")

# NaN 값 제거
valid_mask = ~(pd.isna(dept_y_true) | pd.isna(dept_y_pred))
print(f"   유효한 데이터 개수: {np.sum(valid_mask)}")

if np.sum(valid_mask) > 0:
    dept_y_true_clean = dept_y_true[valid_mask]
    dept_y_pred_clean = dept_y_pred[valid_mask]
    
    ensemble_smape_dept = smape(dept_y_true_clean, dept_y_pred_clean)
    
    print(f"\n🏆 백화점 성능 비교:")
    if not np.isnan(total_type_smape):
        print(f"   백화점 타입별 모델 SMAPE: {total_type_smape:.4f}")
    else:
        print(f"   백화점 타입별 모델 SMAPE: 계산 불가")
        
    if not np.isnan(total_individual_smape):
        print(f"   백화점 개별 모델 SMAPE: {total_individual_smape:.4f}")
    else:
        print(f"   백화점 개별 모델 SMAPE: 계산 불가")
        
    print(f"   백화점 앙상블 모델 SMAPE: {ensemble_smape_dept:.4f}")
    
    # 성능 향상 확인 (NaN이 아닌 경우만)
    if not np.isnan(total_type_smape) and not np.isnan(total_individual_smape):
        improvement_vs_type = total_type_smape - ensemble_smape_dept
        improvement_vs_individual = total_individual_smape - ensemble_smape_dept
        
        print(f"\n📈 백화점 성능 향상:")
        print(f"   vs 타입별 모델: {improvement_vs_type:+.4f} SMAPE")
        print(f"   vs 개별 모델: {improvement_vs_individual:+.4f} SMAPE")
        
        if ensemble_smape_dept < min(total_type_smape, total_individual_smape):
            print("\n🎉 백화점 앙상블이 개별 모델들보다 우수한 성능을 보입니다!")
        else:
            print("\n⚠️ 백화점 앙상블 성능이 기대보다 낮습니다. 가중치 조정을 고려해보세요.")
else:
    print("⚠️ 백화점 성능 계산 중 유효한 데이터가 없습니다.")
    ensemble_smape_dept = float('nan')

print(f"\n✅ 백화점 앙상블 + 기존 결과 결합 완료")
print(f"📊 최종 제출 파일:")
print(f"   - 백화점 건물: 새로운 앙상블 예측값 ({len(dept_test_indices)}개)")
print(f"   - 기타 건물: 기존 앙상블 예측값 ({len(final_submission) - len(dept_test_indices)}개)")


🎯 백화점 앙상블 모델 + 기존 결과 결합 중...
📂 기존 앙상블 결과 로드 중...
✅ 기존 제출 파일 로드 완료: (16800, 2)
🏢 백화점 건물 번호: [18, 19, 27, 29, 32, 34, 40, 45, 54, 59, 63, 73, 74, 79, 88, 95]

📊 백화점 앙상블 가중치:
   개별 건물 모델: 70.0%
   건물 타입별 모델: 30.0%
📊 앙상블 전 데이터 상태 확인:
   개별 모델 예측값 개수: 2688
   타입별 모델 예측값 개수: 2688
   개별 모델 OOF 예측값 개수: 32636
   타입별 모델 OOF 예측값 개수: 32636
   앙상블 후 검증 예측값 개수: 32636
   앙상블 후 테스트 예측값 개수: 2688
📊 백화점 테스트 데이터 인덱스 수: 2688
📊 예측값 교체 과정:
   백화점 테스트 인덱스 범위: 2856 ~ 15959
   앙상블 예측값 인덱스 범위: 2856 ~ 15959
   성공적으로 교체된 예측값: 2688개

📊 성능 계산을 위한 데이터 확인:
   백화점 훈련 인덱스 수: 32636
   실제값 개수: 32636
   예측값 개수: 32636
   유효한 데이터 개수: 32636

🏆 백화점 성능 비교:
   백화점 타입별 모델 SMAPE: 3.6176
   백화점 개별 모델 SMAPE: 3.4604
   백화점 앙상블 모델 SMAPE: 3.3549

📈 백화점 성능 향상:
   vs 타입별 모델: +0.2628 SMAPE
   vs 개별 모델: +0.1055 SMAPE

🎉 백화점 앙상블이 개별 모델들보다 우수한 성능을 보입니다!

✅ 백화점 앙상블 + 기존 결과 결합 완료
📊 최종 제출 파일:
   - 백화점 건물: 새로운 앙상블 예측값 (2688개)
   - 기타 건물: 기존 앙상블 예측값 (14112개)


In [17]:
print("📝 최종 제출 파일 생성 중...")

# 음수값 처리 (전력 소비량은 음수가 될 수 없음)
final_submission["answer"] = np.maximum(final_submission["answer"], 0)

# 최종 제출 파일 저장
final_submission.to_csv('result_csv/ensemble_submission_with_dept_holiday3.csv', index=False)

print("✅ 최종 제출 파일 저장: result_csv/ensemble_submission_with_dept_holiday.csv")




📝 최종 제출 파일 생성 중...
✅ 최종 제출 파일 저장: result_csv/ensemble_submission_with_dept_holiday.csv
